In [6]:
import boto3
import json
import pandas as pd
import awswrangler as wr
from pypfopt.expected_returns import ema_historical_return
from pypfopt import EfficientFrontier, EfficientCVaR
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from botocore.client import ClientError

dynamo_tb = 'PortfolioSubscribers'
my_session = boto3.Session(region_name="us-east-1")
lambda_client = boto3.client('lambda',region_name="us-east-1")
ses_client = boto3.client('sesv2',region_name="us-east-1")

In [41]:
# Portfolio information
symbol = str(input('Enter stocks in your portfolio: '))
method = str(input('Enter portfolio optimization method: '))
total_portfolio_value = float(input('Enter your budget: '))
additional_param = 0

In [ ]:
# aapl,abnb,acva,asle,dcom,dmlp,nuva,verv,tsla

In [42]:
#     min_volatility() :optimizes for minimum volatility
#     max_sharpe() :optimizes for maximal Sharpe ratio (a.k.a the tangency portfolio)
#     max_quadratic_utility() :maximises the quadratic utility, given some risk aversion.
#     efficient_risk() :maximises return for a given target risk
#     efficient_return() :minimises risk for a given target return
#     min_cvar() :minimises the CVaR (minimizing the CVaR more generally requires computing the quantile from a distribution that best fits the data.)

# Porfolio allocation
if hasattr(EfficientFrontier, method):
    if method == 'efficient_risk':
        additional_param  = float(input('Enter the desired maximum volatility of the resulting portfolio: '))
    elif method == 'efficient_return':
        additional_param = float(input('Enter the desired return of the resulting portfolio: '))
        
payload = {
    "symbol":symbol,
    "method":method,
    "additional_param":additional_param,
    "total_portfolio_value":total_portfolio_value
}

response = lambda_client.invoke(
    FunctionName='PortfolioExplore', 
    InvocationType='RequestResponse',
    Payload=json.dumps(payload)
)

result = json.loads(response['Payload'].read())

print(f"{result[4]}")
print (f"Weight of portfolio: {result[0]}")
if method !='min_cvar':
    print (f"Expected return: {round(result[1][0]*100,3)}% \nVolatility: {round(result[1][1]*100,3)}% \nSharpe ratio: {round(result[1][2]*100,3)}%")
else:
    print (f"Expected return: {round(result[1][0]*100,3)}% \nCVaR: {round(result[1][1]*100,3)}%")
print (f"Number of shares for each stock: {result[2]}")
print (f"Leftover: {round(result[3],3)}")


From 2015-12-02 00:00:00 to 2022-10-31 00:00:00
Weight of portfolio: {'dmlp': 0.52489, 'nuva': 0.42431, 'verv': 0.02038, 'tsla': 0.03042}
Expected return: 19.824% 
CVaR: 4.222%
Number of shares for each stock: {'dmlp': 368, 'nuva': 192, 'tsla': 2, 'verv': 11}
Leftover: 169.36


In [ ]:
# Subscription information
user_name = str(input('Enter your name: '))
mail_address = str(input('Enter your mail address: '))
phone_num = int(input('Enter your phone number: '))
portfolio_name = int(input('Enter your portfolio name: '))
portfolio_weight = list(result[0].values())
portfolio_weight = [str(i) for i in portfolio_weight]

In [44]:
# Subscripe to risk_analysis topic or modify porfolio
try:
    wr.dynamodb.put_items(
    items=[{
        'user_name': user_name, 
        'portfolio':portfolio_name,
        'portfolio_weight':portfolio_weight,
        'mail_address':mail_address,
        'phone_num':phone_num,
        'symbol':symbol,
        'method':method,
        'total_portfolio_value':int(total_portfolio_value),
        'additional_param':additional_param
    }],
    table_name=dynamo_tb,
    boto3_session = my_session
    )
except ClientError as e:
    print (e)

# Check if Email identity already exist
try:
    ses_response = ses_client.create_email_identity(
        EmailIdentity=mail_address,
    )
except ClientError as e:
    print (e)
    


In [ ]:
# Unsubscripe to risk_analysis topic
try:
    wr.dynamodb.delete_items(
    items=[{
        'user_name': user_name, 
        'portfolio':portfolio_name
    }],
    table_name=dynamo_tb,
    boto3_session = my_session
    )
except ClientError as e:
    print (e)